In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
!pip install transformers
!pip install pyarabic
!pip install datasets
!pip install mlflow
!pip install sentencepiece==0.1.96
!pip install farasapy==0.0.14

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 6.5 MB 33.3 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 126 kB 5.2 MB/s 
     |████████████████████████████████| 312 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 134 kB 29.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 127 kB 37.1 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 42.6 MB/s 
  Attempting uninstall: urllib3
    Found exis

In [ ]:
import torch
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100000)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
# import Data_Preprocessing as dp
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,pipeline,DataCollatorWithPadding,
                          TrainingArguments)
from datasets import Dataset
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
# from arabert.preprocess import ArabertPreprocessor

In [ ]:
%cd /mydrive/NLP_TASK/

/content/gdrive/My Drive/NLP_TASK


In [ ]:
df = pd.read_csv('preprocessed_df_2.csv')
df

,label,text
0,IQ,بالنهايه ينتفض يغير
1,IQ,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالارهاب
2,IQ,مبين كلامه خليجي
3,IQ,يسلملي مرورك وروحك الحلوه
4,IQ,وين الغيبه اخ محمد
...,...,...
458192,BH,مبسوطين منك الي باسطانا
458193,BH,واله ماينده ابش يختي
458194,BH,شو عملنا لك حنا تهربي منا احنا مساكين ليش بتعملي هيك فينا
458195,BH,اله يبارك وبالعافيه


In [ ]:
mapping = {item:i for i, item in enumerate(df["label"].unique())}
df["label"] = df["label"].apply(lambda x: mapping[x])
df

,label,text
0,0,بالنهايه ينتفض يغير
1,0,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالارهاب
2,0,مبين كلامه خليجي
3,0,يسلملي مرورك وروحك الحلوه
4,0,وين الغيبه اخ محمد
...,...,...
458192,17,مبسوطين منك الي باسطانا
458193,17,واله ماينده ابش يختي
458194,17,شو عملنا لك حنا تهربي منا احنا مساكين ليش بتعملي هيك فينا
458195,17,اله يبارك وبالعافيه


In [ ]:
mapping

{'AE': 16,
 'BH': 17,
 'DZ': 10,
 'EG': 11,
 'IQ': 0,
 'JO': 6,
 'KW': 13,
 'LB': 12,
 'LY': 1,
 'MA': 7,
 'OM': 14,
 'PL': 3,
 'QA': 2,
 'SA': 8,
 'SD': 15,
 'SY': 4,
 'TN': 5,
 'YE': 9}

In [ ]:
X_train,X_test =train_test_split(df,test_size=0.2,stratify=df['label'],random_state=0)

In [ ]:
model_name="aubmindlab/bert-base-arabertv2"

In [ ]:
X_train_1 = Dataset.from_pandas(X_train)
X_test_1 = Dataset.from_pandas(X_test)

In [25]:
X_train_1

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 366557
})

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=18)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [18]:
def preprocess_function(examples):
    return tokenizer(examples["text"],max_length=280,padding='max_length',truncation=True)

In [19]:
X_train_tokenized = X_train_1.map(preprocess_function)
X_test_tokenized = X_test_1.map(preprocess_function)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [32]:
X_train_tokenized

Dataset({
    features: ['label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 366557
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)

  macro_f1 = f1_score(p.label_ids,preds,average='macro')

  acc = accuracy_score(p.label_ids,preds)
  return {       
      'macro_f1' : macro_f1,
      'accuracy': acc
  }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end = True,
    evaluation_strategy = 'steps',
    save_strategy = 'steps',
    eval_steps =8000,
    save_steps=8000,
    metric_for_best_model = 'macro_f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train_tokenized,
    eval_dataset=X_test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
trainer.train(resume_from_checkpoint=True)

Loading model from ./results/checkpoint-40000).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 366557
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 45820
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 40000
  Will skip the first 1 epochs then the first 17090 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/17090 [00:00<?, ?it/s]

Trainer is attempting to log a value of "{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7', 8: 'LABEL_8', 9: 'LABEL_9', 10: 'LABEL_10', 11: 'LABEL_11', 12: 'LABEL_12', 13: 'LABEL_13', 14: 'LABEL_14', 15: 'LABEL_15', 16: 'LABEL_16', 17: 'LABEL_17'}" for key "id2label" as a parameter. MLflow's log_param() only accepts values no longer than 250 characters so we dropped this attribute.
Trainer is attempting to log a value of "{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5, 'LABEL_6': 6, 'LABEL_7': 7, 'LABEL_8': 8, 'LABEL_9': 9, 'LABEL_10': 10, 'LABEL_11': 11, 'LABEL_12': 12, 'LABEL_13': 13, 'LABEL_14': 14, 'LABEL_15': 15, 'LABEL_16': 16, 'LABEL_17': 17}" for key "label2id" as a parameter. MLflow's log_param() only accepts values no longer than 250 characters so we dropped this attribute.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-40000 (score: 0.478422856772118).


TrainOutput(global_step=45820, training_loss=0.18928631000360616, metrics={'train_runtime': 8581.4567, 'train_samples_per_second': 85.43, 'train_steps_per_second': 5.339, 'total_flos': 7.096423527106704e+16, 'train_loss': 0.18928631000360616, 'epoch': 2.0})

In [ ]:
trainer.evaluate(X_test_tokenized)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 91640
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.5099628982976866,
 'eval_loss': 1.5411642789840698,
 'eval_macro_f1': 0.478422856772118,
 'eval_runtime': 3056.3653,
 'eval_samples_per_second': 29.983,
 'eval_steps_per_second': 1.874}

In [ ]:
inv_label_map = inv_label_map = { v:k for k, v in mapping.items()}
print(inv_label_map)
trainer.model.config.label2id = mapping
trainer.model.config.id2label = inv_label_map
trainer.save_model("output_dir")
# train_dataset.tokenizer.save_pretrained("output_dir")
trainer.tokenizer.save_pretrained("output_dir")

Saving model checkpoint to output_dir
Configuration saved in output_dir/config.json


{0: 'IQ', 1: 'LY', 2: 'QA', 3: 'PL', 4: 'SY', 5: 'TN', 6: 'JO', 7: 'MA', 8: 'SA', 9: 'YE', 10: 'DZ', 11: 'EG', 12: 'LB', 13: 'KW', 14: 'OM', 15: 'SD', 16: 'AE', 17: 'BH'}


Model weights saved in output_dir/pytorch_model.bin
tokenizer config file saved in output_dir/tokenizer_config.json
Special tokens file saved in output_dir/special_tokens_map.json
tokenizer config file saved in output_dir/tokenizer_config.json
Special tokens file saved in output_dir/special_tokens_map.json


('output_dir/tokenizer_config.json',
 'output_dir/special_tokens_map.json',
 'output_dir/vocab.txt',
 'output_dir/added_tokens.json',
 'output_dir/tokenizer.json')

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="output_dir")

loading configuration file output_dir/config.json
Model config BertConfig {
  "_name_or_path": "output_dir",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "IQ",
    "1": "LY",
    "2": "QA",
    "3": "PL",
    "4": "SY",
    "5": "TN",
    "6": "JO",
    "7": "MA",
    "8": "SA",
    "9": "YE",
    "10": "DZ",
    "11": "EG",
    "12": "LB",
    "13": "KW",
    "14": "OM",
    "15": "SD",
    "16": "AE",
    "17": "BH"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "AE": 16,
    "BH": 17,
    "DZ": 10,
    "EG": 11,
    "IQ": 0,
    "JO": 6,
    "KW": 13,
    "LB": 12,
    "LY": 1,
    "MA": 7,
    "OM": 14,
    "PL": 3,
    "QA": 2,
    "SA": 8,
    "SD": 15,
    "SY": 4,
    "TN": 5,
    "YE": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mo